**BiSeNet Training**

In [ ]:
%cd /content
!git clone https://github.com/m-depascale/AnomalySegmentation_AML

In [ ]:
!pip install ood-metrics
!pip install visdom
!pip install numpy
!pip install matplotlib
!pip install Pillow
!pip install torchvision

In [ ]:
!python -m pip install cityscapesscripts
!python -m pip install cityscapesscripts[gui]

In [ ]:
from cityscapesscripts.download import downloader
print(dir(downloader))
import os

# Esegui il login
session = downloader.login()
names = downloader.list_available_packages(session=session)
print(names)


destination_path = "/content/Datasets/Cityscapes"
os.makedirs(destination_path, exist_ok=True)

downloader.download_packages(session=session, package_names=['leftImg8bit_trainvaltest.zip', 'gtFine_trainvaltest.zip'], destination_path=destination_path)

In [ ]:
!unzip /content/Datasets/Cityscapes/leftImg8bit_trainvaltest.zip -d /content/Datasets/Cityscapes/

In [ ]:
!unzip /content/Datasets/Cityscapes/gtFine_trainvaltest.zip -d /content/Datasets/Cityscapes/

In [ ]:
os.environ['CITYSCAPES_DATASET'] = '/content/Datasets/Cityscapes'
from cityscapesscripts.preparation.createTrainIdLabelImgs import main as convertor
convertor()

Processing 5000 annotation files
Progress: 100.0 % 

In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
%cd /content
!gdown 1r2eFANvSlcUjxcerjC8l6dRa0slowMpx

In [ ]:
!unzip /content/Validation_Dataset.zip -d /content

In [ ]:
!cp /content/AnomalySegmentation_AML/eval/bisenetv1.py /content/AnomalySegmentation_AML/train/bisenetv1.py

In [ ]:
!cp /content/AnomalySegmentation_AML/eval/resnet.py /content/AnomalySegmentation_AML/train/resnet.py

In [ ]:
%cd /content/AnomalySegmentation_AML/trained_models

/content/AnomalySegmentation_AML/trained_models


In [ ]:
# Extract the first part
!unrar x /content/AnomalySegmentation_AML/trained_models/bisenet_finetuned.part1.rar

# Extract the second part
!unrar x /content/AnomalySegmentation_AML/trained_models/bisenet_finetuned.part2.rar

#Select "All" when colab asks you something.
#Then you will find bisenet_finetuned.pth on /content/bisenet_finetuned.pth

In [ ]:
%cd /content/AnomalySegmentation_AML/eval

/content/AnomalySegmentation_AML/eval


In [ ]:
!python eval_iou.py --datadir "/content/Datasets/Cityscapes" --subset 'val' --loadWeights='bisenet_finetuned.pth' --loadModel='bisenetv1.py'

In [ ]:
!python evalAnomaly.py --method "VOID" --loadModel='bisenetv1.py' --loadWeights='bisenet_finetuned.pth' --input "/content/Validation_Dataset/RoadObsticle21/images/*.webp"

Loading model: ../trained_models/bisenetv1.py
Loading weights: ../trained_models/bisenet_finetuned.pth
--- SAVED PRETRAINED PARAMS ---- 
Model and weights LOADED successfully
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
AUPRC score: 0.6374021338018769
FPR@TPR95: 98.7324336447037


In [ ]:
!python evalAnomaly.py --method "VOID" --loadModel='bisenetv1.py' --loadWeights='bisenet_finetuned.pth' --input "/content/Validation_Dataset/RoadAnomaly21/images/*.png"

Loading model: ../trained_models/bisenetv1.py
Loading weights: ../trained_models/bisenet_finetuned.pth
--- SAVED PRETRAINED PARAMS ---- 
Model and weights LOADED successfully
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
AUPRC score: 19.868139792749435
FPR@TPR95: 94.76721506874186


In [ ]:
!python evalAnomaly.py --method "VOID" --loadModel='bisenetv1.py' --loadWeights='bisenet_finetuned.pth' --input "/content/Validation_Dataset/RoadAnomaly/images/*.jpg"

python3: can't open file '/content/AnomalySegmentation_AML/train/evalAnomaly.py': [Errno 2] No such file or directory


In [ ]:
!python evalAnomaly.py --method "VOID" --loadModel='bisenetv1.py' --loadWeights='bisenet_finetuned.pth' --input "/content/Validation_Dataset/fs_static/images/*.jpg"

Loading model: ../trained_models/bisenetv1.py
Loading weights: ../trained_models/bisenet_finetuned.pth
--- SAVED PRETRAINED PARAMS ---- 
Model and weights LOADED successfully
AUPRC score: 1.5074097402817843
FPR@TPR95: 97.86444374061732


In [ ]:
!python evalAnomaly.py --method "VOID" --loadModel='bisenetv1.py' --loadWeights='bisenet_finetuned.pth' --input "/content/Validation_Dataset/FS_LostFound_full/images/*.png"

Loading model: ../trained_models/bisenetv1.py
Loading weights: ../trained_models/bisenet_finetuned.pth
--- SAVED PRETRAINED PARAMS ---- 
Model and weights LOADED successfully
^C


In [ ]:
%cd /content/AnomalySegmentation_AML/train

/content/AnomalySegmentation_AML/train


In [ ]:
import os
import random
import time
import numpy as np
import torch
import math

from PIL import Image, ImageOps
from argparse import ArgumentParser

from torch.optim import SGD, Adam, lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad
from torchvision.transforms import ToTensor, ToPILImage

from dataset import VOC12,cityscapes
from transform import Relabel, ToLabel, Colorize
from visualize import Dashboard

import importlib
from iouEval import iouEval, getColorEntry

from shutil import copyfile

NUM_CHANNELS = 3
NUM_CLASSES = 20 #pascal=22, cityscapes=20

color_transform = Colorize(NUM_CLASSES)
image_transform = ToPILImage()

#Augmentations - different function implemented to perform random augments on both image and target
class MyCoTransform(object):
    def __init__(self, enc, augment=True, height=512):
        self.enc=enc
        self.augment = augment
        self.height = height
        pass
    def __call__(self, input, target):
        # do something to both images
        input =  Resize(self.height, Image.BILINEAR)(input)
        target = Resize(self.height, Image.NEAREST)(target)

        if(self.augment):
            # Random hflip
            hflip = random.random()
            if (hflip < 0.5):
                input = input.transpose(Image.FLIP_LEFT_RIGHT)
                target = target.transpose(Image.FLIP_LEFT_RIGHT)

            #Random translation 0-2 pixels (fill rest with padding
            transX = random.randint(-2, 2)
            transY = random.randint(-2, 2)

            input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
            target = ImageOps.expand(target, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
            target = target.crop((0, 0, target.size[0]-transX, target.size[1]-transY))

        input = ToTensor()(input)
        if (self.enc):
            target = Resize(int(self.height/8), Image.NEAREST)(target)
        target = ToLabel()(target)
        target = Relabel(255, 19)(target)

        return input, target


class CrossEntropyLoss2d(torch.nn.Module):

    def __init__(self, weight=None):
        super().__init__()

        self.loss = torch.nn.NLLLoss2d(weight)

    def forward(self, outputs, targets):
        return self.loss(torch.nn.functional.log_softmax(outputs, dim=1), targets)


def train(args, model, enc=False):
    best_acc = 0

    #TODO: calculate weights by processing dataset histogram (now its being set by hand from the torch values)
    #create a loder to run all images and calculate histogram of labels, then create weight array using class balancing

    weight = torch.ones(NUM_CLASSES)
    if (enc):
        weight[0] = 2.3653597831726
        weight[1] = 4.4237880706787
        weight[2] = 2.9691488742828
        weight[3] = 5.3442072868347
        weight[4] = 5.2983593940735
        weight[5] = 5.2275490760803
        weight[6] = 5.4394111633301
        weight[7] = 5.3659925460815
        weight[8] = 3.4170460700989
        weight[9] = 5.2414722442627
        weight[10] = 4.7376127243042
        weight[11] = 5.2286224365234
        weight[12] = 5.455126285553
        weight[13] = 4.3019247055054
        weight[14] = 5.4264230728149
        weight[15] = 5.4331531524658
        weight[16] = 5.433765411377
        weight[17] = 5.4631009101868
        weight[18] = 5.3947434425354
    else:
        weight[0] = 2.8149201869965
        weight[1] = 6.9850029945374
        weight[2] = 3.7890393733978
        weight[3] = 9.9428062438965
        weight[4] = 9.7702074050903
        weight[5] = 9.5110931396484
        weight[6] = 10.311357498169
        weight[7] = 10.026463508606
        weight[8] = 4.6323022842407
        weight[9] = 9.5608062744141
        weight[10] = 7.8698215484619
        weight[11] = 9.5168733596802
        weight[12] = 10.373730659485
        weight[13] = 6.6616044044495
        weight[14] = 10.260489463806
        weight[15] = 10.287888526917
        weight[16] = 10.289801597595
        weight[17] = 10.405355453491
        weight[18] = 10.138095855713

    weight[19] = 0

    assert os.path.exists(args['datadir']), "Error: datadir (dataset directory) could not be loaded"

    co_transform = MyCoTransform(enc, augment=True, height=args['height'])#1024)
    co_transform_val = MyCoTransform(enc, augment=False, height=args['height'])#1024)
    dataset_train = cityscapes(args['datadir'], co_transform, 'train')
    dataset_val = cityscapes(args['datadir'], co_transform_val, 'val')

    loader = DataLoader(dataset_train, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=True)
    loader_val = DataLoader(dataset_val, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=False)

    if args['cuda']:
        weight = weight.cuda()
    criterion = CrossEntropyLoss2d(weight)
    print(type(criterion))

    savedir = args['savedir']

    if (enc):
        automated_log_path = savedir + "/automated_log_encoder.txt"
        modeltxtpath = savedir + "/model_encoder.txt"
    else:
        automated_log_path = savedir + "/automated_log.txt"
        modeltxtpath = savedir + "/model.txt"

    if (not os.path.exists(automated_log_path)):    #dont add first line if it exists
        with open(automated_log_path, "a") as myfile:
            myfile.write("Epoch\t\tTrain-loss\t\tTest-loss\t\tTrain-IoU\t\tTest-IoU\t\tlearningRate")

    with open(modeltxtpath, "w") as myfile:
        myfile.write(str(model))


    #optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=2e-4)     ## scheduler 1
    optimizer = Adam(model.parameters(), 5e-4, (0.9, 0.999),  eps=1e-08, weight_decay=1e-4)      ## scheduler 2

    start_epoch = 1
    if args['resume']:
        #Must load weights, optimizer, epoch and best value.
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'

        assert os.path.exists(filenameCheckpoint), "Error: resume option was used but checkpoint was not found in folder"
        checkpoint = torch.load(filenameCheckpoint)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        print("=> Loaded checkpoint at epoch {})".format(checkpoint['epoch']))

    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5) # set up scheduler     ## scheduler 1
    lambda1 = lambda epoch: pow((1-((epoch-1)/args['num_epochs'])),0.9)  ## scheduler 2
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)                             ## scheduler 2

    if args['visualize'] and args['steps_plot'] > 0:
        board = Dashboard(8097)

    for epoch in range(start_epoch, args['num_epochs']+1):
        print("----- TRAINING - EPOCH", epoch, "-----")
        #print(weight)
        scheduler.step(epoch)    ## scheduler 2

        epoch_loss = []
        time_train = []

        doIouTrain = args['iouTrain']
        doIouVal =  args['iouVal']

        if (doIouTrain):
            iouEvalTrain = iouEval(NUM_CLASSES)

        usedLr = 0
        for param_group in optimizer.param_groups:
            print("LEARNING RATE: ", param_group['lr'])
            usedLr = float(param_group['lr'])

        model.train()
        for step, (images, labels) in enumerate(loader):

            start_time = time.time()

            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images)
            targets = Variable(labels)
            outputs, *_ = model(inputs)
            #print(outputs.shape)


            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.data.item())
            time_train.append(time.time() - start_time)

            if (doIouTrain):
                iouEvalTrain.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            #print(outputs.size())
            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data

                board.image(image, f'input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_train) / len(time_train) / args['batch_size']))


        average_epoch_loss_train = sum(epoch_loss) / len(epoch_loss)

        iouTrain = 0
        if (doIouTrain):
            iouTrain, iou_classes = iouEvalTrain.getIoU()
            iouStr = getColorEntry(iouTrain)+'{:0.2f}'.format(iouTrain*100) + '\033[0m'
            print ("EPOCH IoU on TRAIN set: ", iouStr, "%")

        #Validate on 500 val images after each epoch of training
        print("----- VALIDATING - EPOCH", epoch, "-----")
        model.eval()
        epoch_loss_val = []
        time_val = []

        if (doIouVal):
            iouEvalVal = iouEval(NUM_CLASSES)

        for step, (images, labels) in enumerate(loader_val):
            start_time = time.time()
            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
            targets = Variable(labels, volatile=True)
            outputs, *_ = model(inputs)

            loss = criterion(outputs, targets[:, 0])
            epoch_loss_val.append(loss.data.item())
            time_val.append(time.time() - start_time)


            #Add batch to calculate TP, FP and FN for iou estimation
            if (doIouVal):
                iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data
                board.image(image, f'VAL input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'VAL target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss_val) / len(epoch_loss_val)
                print(f'VAL loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_val) / len(time_val) / args['batch_size']))


        average_epoch_loss_val = sum(epoch_loss_val) / len(epoch_loss_val)

        iouVal = 0
        if (doIouVal):
            iouVal, iou_classes = iouEvalVal.getIoU()
            iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
            print ("EPOCH IoU on VAL set: ", iouStr, "%")


        if iouVal == 0:
            current_acc = -average_epoch_loss_val
        else:
            current_acc = iouVal
        is_best = current_acc > best_acc
        best_acc = max(current_acc, best_acc)
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
            filenameBest = savedir + '/model_best_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'
            filenameBest = savedir + '/model_best.pth.tar'
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': str(model),
            'state_dict': model.state_dict(),
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filenameCheckpoint, filenameBest)

        #SAVE MODEL AFTER EPOCH
        if (enc):
            filename = f'{savedir}/model_encoder-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_encoder_best.pth'
        else:
            filename = f'{savedir}/model-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_best.pth'
        if args['epochs_save'] > 0 and step > 0 and step % args['epochs_save'] == 0:
            torch.save(model.state_dict(), filename)
            print(f'save: {filename} (epoch: {epoch})')
        if (is_best):
            torch.save(model.state_dict(), filenamebest)
            print(f'save: {filenamebest} (epoch: {epoch})')
            if (not enc):
                with open(savedir + "/best.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))
            else:
                with open(savedir + "/best_encoder.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))

        #SAVE TO FILE A ROW WITH THE EPOCH RESULT (train loss, val loss, train IoU, val IoU)
        #Epoch		Train-loss		Test-loss	Train-IoU	Test-IoU		learningRate
        with open(automated_log_path, "a") as myfile:
            myfile.write("\n%d\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.8f" % (epoch, average_epoch_loss_train, average_epoch_loss_val, iouTrain, iouVal, usedLr ))

    return(model)   #return model (convenience for encoder-decoder training)

def save_checkpoint(state, is_best, filenameCheckpoint, filenameBest):
    torch.save(state, filenameCheckpoint)
    if is_best:
        print ("Saving model as best")
        torch.save(state, filenameBest)

In [ ]:
from bisenetv1 import BiSeNetV1

model = BiSeNetV1(NUM_CLASSES)

In [ ]:
import logging
import logging.handlers
import sys


savedir = f'../save/BiSeNet_finetuned'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 8,
    'cuda' : True,
    'savedir' : savedir,
    'resume': False,
    'num_epochs': 20,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': False
}


bisenet_model = BiSeNetV1(NUM_CLASSES)

keys = ['conv_out.conv_out',
        'conv_out16.conv_out',
        'conv_out32.conv_out']

new_dict = torch.load('/content/AnomalySegmentation_AML/trained_models/bisenet_finetuned.pth')

model.load_state_dict(new_dict)
model.cuda()

for name, param in model.named_parameters():
  splits = name.split('.')
  if str(splits[0]+'.'+splits[1]) in keys:
    param.requires_grad = True
  else:
    param.requires_grad = True


bisenet_fine_tuned = train(args, model)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
LEARNING RATE:  0.0005


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:222: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is 

loss: 0.1899 (epoch: 1, step: 0) // Avg time/img: 0.0430 s
loss: 0.2393 (epoch: 1, step: 50) // Avg time/img: 0.0212 s
loss: 0.2666 (epoch: 1, step: 100) // Avg time/img: 0.0211 s
loss: 0.2939 (epoch: 1, step: 150) // Avg time/img: 0.0212 s
loss: 0.303 (epoch: 1, step: 200) // Avg time/img: 0.0211 s
loss: 0.2985 (epoch: 1, step: 250) // Avg time/img: 0.0211 s
loss: 0.2991 (epoch: 1, step: 300) // Avg time/img: 0.0212 s
loss: 0.2966 (epoch: 1, step: 350) // Avg time/img: 0.0212 s
----- VALIDATING - EPOCH 1 -----


<ipython-input-95-1e75ccddfce5>:277: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-95-1e75ccddfce5>:278: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.2714 (epoch: 1, step: 0) // Avg time/img: 0.0123 s
VAL loss: 0.4081 (epoch: 1, step: 50) // Avg time/img: 0.0108 s
EPOCH IoU on VAL set:  52.88 %
Saving model as best
save: ../save/BiSeNet_finetuned/model_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
LEARNING RATE:  0.00047744269081074987
loss: 0.2519 (epoch: 2, step: 0) // Avg time/img: 0.0244 s
loss: 0.299 (epoch: 2, step: 50) // Avg time/img: 0.0211 s
loss: 0.2837 (epoch: 2, step: 100) // Avg time/img: 0.0212 s
loss: 0.286 (epoch: 2, step: 150) // Avg time/img: 0.0212 s
loss: 0.2812 (epoch: 2, step: 200) // Avg time/img: 0.0212 s
loss: 0.2777 (epoch: 2, step: 250) // Avg time/img: 0.0212 s
loss: 0.2824 (epoch: 2, step: 300) // Avg time/img: 0.0212 s
loss: 0.278 (epoch: 2, step: 350) // Avg time/img: 0.0212 s
----- VALIDATING - EPOCH 2 -----
VAL loss: 0.226 (epoch: 2, step: 0) // Avg time/img: 0.0132 s
VAL loss: 0.3522 (epoch: 2, step: 50) // Avg time/img: 0.0107 s
EPOCH IoU on VAL set:  57.38 %
Saving model as best


In [ ]:
savedir = f'../save/BiSeNet_finetuned'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 8,
    'cuda' : True,
    'savedir' : savedir,
    'resume': False,
    'num_epochs': 30,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': True
}


bisenet_fine_tuned = train(args, model)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:222: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is 

=> Loaded checkpoint at epoch 21)
----- TRAINING - EPOCH 21 -----
LEARNING RATE:  0.00018602052900565077
loss: 0.1141 (epoch: 21, step: 0) // Avg time/img: 0.0259 s
loss: 0.1211 (epoch: 21, step: 50) // Avg time/img: 0.0216 s
loss: 0.1265 (epoch: 21, step: 100) // Avg time/img: 0.0214 s
loss: 0.1293 (epoch: 21, step: 150) // Avg time/img: 0.0214 s
loss: 0.1405 (epoch: 21, step: 200) // Avg time/img: 0.0214 s
loss: 0.1524 (epoch: 21, step: 250) // Avg time/img: 0.0214 s
loss: 0.168 (epoch: 21, step: 300) // Avg time/img: 0.0214 s
loss: 0.1726 (epoch: 21, step: 350) // Avg time/img: 0.0214 s
----- VALIDATING - EPOCH 21 -----


<ipython-input-95-1e75ccddfce5>:277: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-95-1e75ccddfce5>:278: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.2243 (epoch: 21, step: 0) // Avg time/img: 0.0136 s
VAL loss: 0.34 (epoch: 21, step: 50) // Avg time/img: 0.0107 s
EPOCH IoU on VAL set:  58.93 %
----- TRAINING - EPOCH 22 -----
LEARNING RATE:  0.00016919173095082495
loss: 0.2138 (epoch: 22, step: 0) // Avg time/img: 0.0250 s
loss: 0.1795 (epoch: 22, step: 50) // Avg time/img: 0.0218 s
loss: 0.1745 (epoch: 22, step: 100) // Avg time/img: 0.0215 s
loss: 0.1719 (epoch: 22, step: 150) // Avg time/img: 0.0214 s
loss: 0.1698 (epoch: 22, step: 200) // Avg time/img: 0.0214 s
loss: 0.1663 (epoch: 22, step: 250) // Avg time/img: 0.0213 s
loss: 0.1649 (epoch: 22, step: 300) // Avg time/img: 0.0213 s
loss: 0.1637 (epoch: 22, step: 350) // Avg time/img: 0.0213 s
----- VALIDATING - EPOCH 22 -----
VAL loss: 0.1946 (epoch: 22, step: 0) // Avg time/img: 0.0083 s
VAL loss: 0.3069 (epoch: 22, step: 50) // Avg time/img: 0.0103 s
EPOCH IoU on VAL set:  64.72 %
----- TRAINING - EPOCH 23 -----
LEARNING RATE:  0.00015217449296258857
loss: 0.1428 

In [ ]:
for name, param in model.named_parameters():
  splits = name.split('.')
  if str(splits[0]+'.'+splits[1]) in keys:
    param.requires_grad = True
  else:
    param.requires_grad = False

In [ ]:
savedir = f'../save/BiSeNet_finetuned'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 8,
    'cuda' : True,
    'savedir' : savedir,
    'resume': False,
    'num_epochs': 35,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': True
}


bisenet_fine_tuned = train(args, model)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
=> Loaded checkpoint at epoch 31)
----- TRAINING - EPOCH 31 -----
LEARNING RATE:  8.677243171707623e-05


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:222: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is 

loss: 0.1205 (epoch: 31, step: 0) // Avg time/img: 0.0112 s
loss: 0.1057 (epoch: 31, step: 50) // Avg time/img: 0.0125 s
loss: 0.1058 (epoch: 31, step: 100) // Avg time/img: 0.0122 s
loss: 0.1053 (epoch: 31, step: 150) // Avg time/img: 0.0121 s
loss: 0.1048 (epoch: 31, step: 200) // Avg time/img: 0.0123 s
loss: 0.1039 (epoch: 31, step: 250) // Avg time/img: 0.0123 s
loss: 0.1041 (epoch: 31, step: 300) // Avg time/img: 0.0123 s
loss: 0.1043 (epoch: 31, step: 350) // Avg time/img: 0.0123 s
----- VALIDATING - EPOCH 31 -----


<ipython-input-95-1e75ccddfce5>:277: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-95-1e75ccddfce5>:278: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.1857 (epoch: 31, step: 0) // Avg time/img: 0.0114 s
VAL loss: 0.3214 (epoch: 31, step: 50) // Avg time/img: 0.0113 s
EPOCH IoU on VAL set:  67.54 %
Saving model as best
save: ../save/BiSeNet_finetuned/model_best.pth (epoch: 31)
----- TRAINING - EPOCH 32 -----
LEARNING RATE:  7.098437398420578e-05
loss: 0.08686 (epoch: 32, step: 0) // Avg time/img: 0.0144 s
loss: 0.1032 (epoch: 32, step: 50) // Avg time/img: 0.0125 s
loss: 0.1031 (epoch: 32, step: 100) // Avg time/img: 0.0124 s
loss: 0.1039 (epoch: 32, step: 150) // Avg time/img: 0.0123 s
loss: 0.1045 (epoch: 32, step: 200) // Avg time/img: 0.0120 s
loss: 0.1047 (epoch: 32, step: 250) // Avg time/img: 0.0120 s
loss: 0.1047 (epoch: 32, step: 300) // Avg time/img: 0.0119 s
loss: 0.1046 (epoch: 32, step: 350) // Avg time/img: 0.0118 s
----- VALIDATING - EPOCH 32 -----
VAL loss: 0.1853 (epoch: 32, step: 0) // Avg time/img: 0.0115 s
VAL loss: 0.3245 (epoch: 32, step: 50) // Avg time/img: 0.0108 s
EPOCH IoU on VAL set:  67.56 %
Sa